# Feature Selection using BPSO for SDP

installations

import libraries 

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import os
import pyswarms as ps

%load_ext autoreload
%autoreload 2
%matplotlib inline

### The methods 

Dataset prepration

In [3]:
def data_prepration(file):
    df=pd.read_csv(file)
    if( 'version' in file):
        X=np.array(df.drop(['bug','name.1','name', 'version'], axis=1))
    else:
        X=np.array(df.drop('bug', axis=1))
    y=np.array(df.bug)
    
    return X,y

plot data per feature (each class with different color)

In [4]:
def plot_features(features,labels):
    df = pd.DataFrame(features)
    df['labels'] = pd.Series(labels)

    sns.pairplot(df, hue='labels' , diag_kind='hist')

fitness function for each particle

In [5]:
def f_per_particle(m):
    if(clas=='NB'):
        classifier = GaussianNB()
    elif(clas=='SVM linear'):
        classifier = svm.SVC(kernel='linear')
    elif(clas=='KNN'):
        classifier = KNeighborsClassifier(n_neighbors=5)
    elif(clas=='SVM RBF'):
        classifier = svm.SVC(kernel='rbf')
    """Computes for the fitness function per particle

    Inputs
    ------
    m : numpy.ndarray
        Binary mask that can be obtained from BinaryPSO, will
        be used to mask features.

    Returns
    -------
    numpy.ndarray
        Computed finess function
    """
    
    # Get the subset of the features from the binary mask
    if np.count_nonzero(m) == 0:
        X_subset = X_train
        X_subset_test= X_test
    else:
        X_subset = X_train[:,m==1]
        X_subset_test= X_test[:,m==1]
        
    
    # Perform classification and compute the fitness function
    classifier.fit(X_subset, y_train)
    j = (classifier.predict(X_subset_test) != y_test).mean() 
#     j=np.count_nonzero(m)

    return j

Define fitness function for whole swarm

In [6]:
def f(x):
    """Higher-level method to do classification in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [f_per_particle(x[i]) for i in range(n_particles)]
    
    return np.array(j)

select the features from the set of features X using BPSO on the specfied parameters

In [7]:
def select_features(nparticles):
  
    total_features = X.shape[1]
    
    dimensions = total_features
    
    optimizer = ps.discrete.BinaryPSO(n_particles=nparticles, dimensions=dimensions, options=options)
    
    # Perform optimization
    cost, pos = optimizer.optimize(f, iters=iters)
    
    
    print("cost: "+str(cost))
    fr.write(clas+"-"+ds+" dataset:\ncost= "+str(cost)+"\nNumber of selected features:"+str(len([i for i in pos if i==1]))+"\nSelected features:"+str(pos))
    return cost, pos

Train clssifier on the selected features (corrposining to pos which =1)

In [13]:
def AUC(y_true,y_pred):
    
    from sklearn.metrics import roc_curve, auc
    from sklearn.preprocessing import label_binarize
    from sklearn.multiclass import OneVsRestClassifier
    from scipy import interp

    from sklearn.metrics import roc_curve  
    from sklearn.metrics import roc_auc_score ,auc
    from  sklearn.preprocessing import label_binarize 
    
    classes=list(set(y))
    y_test = label_binarize(y_true, classes=classes)
    y_score = label_binarize(y_pred, classes=classes)

    n_classes = len(classes)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test[:,i], y_score[:,i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    #auc = roc_auc_score(y,y_pred,average='micro')  
      
    

    # Compute macro-average ROC curve and ROC area

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
    
    
    return roc_auc,tpr,fpr

In [8]:
def train_and_test(clas,X_train,X_test,y_train,y_test,pos, fr, fnum, filexs):   
    
    if(clas=='NB'):
        classifier = GaussianNB()
    elif(clas=='SVM linear'):
        classifier = svm.SVC(kernel='linear')
    elif(clas=='KNN'):
        classifier = KNeighborsClassifier(n_neighbors=5)
    elif(clas=='SVM RBF'):
        classifier = svm.SVC(kernel='rbf')
        
    # Get the selected features from the final positions
    X_selected_features = X_train[:,pos==1]  # subset
   
    # Perform classification and store performance in P
    classifier.fit(X_selected_features,y_train)
    selected_test= X_test[:,pos==1]
    
    y_pred= classifier.predict(selected_test)    
    report=classification_report(y_test,y_pred)
    subset_performance = (y_pred== y_test).mean()
    
    print(len(list(set(y))))
    if(len(list(set(y)))>2):
        print('multi')
        matrix = multilabel_confusion_matrix(y_test, y_pred)
        tn = matrix[:, 0, 0]
        tp = matrix[:, 1, 1]
        fn = matrix[:, 1, 0]
        fp = matrix[:, 0, 1]
        G_mean= math.sqrt((tp.mean()/(tp.mean()+fn.mean()))*(tn.mean()/(fp.mean()+tn.mean())))
    else:
        print('binary')
        matrix= confusion_matrix(y_test,y_pred, labels=list(set(y)))
        tn, fp, fn, tp = matrix.ravel()
        G_mean= math.sqrt((tp/(tp+fn))*(tn/(fp+tn)))
    subset_performance = (y_pred== y_test).mean()
    ac=accuracy_score(y_test,y_pred)
    prec=precision_score(y_test,y_pred,average='weighted')
    rc=recall_score(y_test,y_pred,average='weighted')
    F_Score = (2 * rc * prec) / (rc + prec)  
    auc=AUC(y_test,y_pred)[0]["micro"]
    
    fr.write("accuracy_score= {}\n".format(ac))
    fr.write("precision_score= {}\n".format(prec))
    fr.write("recall_score= {}\n".format(rc))
    fr.write("F_Score= {}\n".format(F_Score))
    fr.write("auc= {}\n".format(auc))
    fr.write("G_mean= {}\n".format(G_mean))
    
    
    
    fr.write("confusion_matrix=\n{}\n".format(matrix))
    fr.write("labels {}\n".format(set(y)))
    fr.write("True Negative= {}\n".format(tn))
    fr.write("True Positive= {}\n".format(tp))
    fr.write("Flase Negative= {}\n".format(fn))
    fr.write("False Positive= {}\n".format(fp))

    # Compute performance
#     subset_performance = (classifier.predict(selected_test) == y_test).mean()
    fr.write("\nsubset_performance:"+str(subset_performance)+"\n")
    fr.write("\nclassification_report:\n"+str(report)+"\n")
    
    print('Subset performance: %.3f' % (subset_performance))
    
    
    filexs.loc[fnum] =[ds,prec,ac,rc,F_Score,auc,G_mean]

## The main cell 

here the classifier is created, options, number of iterations are set, and other methods are called

each results for each dataset are stored in results folder with file of the classifier name

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, precision_score, recall_score, accuracy_score, f1_score
import math 

In [10]:
root_folder="D:\\SDP\\Dataset Dr.Sultan\\selected DS"
files = [os.path.join(root_folder, x) for x in os.listdir(root_folder)]

In [14]:
import time


# from sklearn.naive_bayes import GaussianNB
# classifier = GaussianNB()
# clas='NB'

# from sklearn import svm
# classifier = svm.SVC(kernel='linear')
# clas='SVM linear'

# from sklearn import svm
# classifier = svm.SVC(kernel='rbf')
# clas='SVM RBF'

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
clas='KNN'

# from sklearn import linear_model
# classifier = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto')
# clas='Linear Regression'

options = {'c1': 0.6, 'c2': 0.3, 'w': 0.4, 'k':20, 'p':2}
# k : number of neighbors to be considered. Must be a positive integer less than n_partcles 
# p: the Minkowski p-norm to use. 1 is the sum-of-absolute values (or L1 distance) while 2 is the Euclidean (or L2) distance
iters= 100



fnum=-1
filexs=pd.DataFrame(columns=["Dataset", "precision", "Accuracy","Recall","F-measure","AUC","G-mean"])

for file in files [0:1]:
    fnum+=1
    start= time.time()
    
    ds=file.split("\\")[4][0:-4]
    fr = open("C:/Users/Afnan/SDP/SDP/Test/BPSO/"+clas+"/{}.txt".format(ds), "a")
    fr.write('parameters: '+str(options)+"\n")
    
    X,y = data_prepration(file)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    cost, pos = select_features(20)
    train_and_test(clas,X_train,X_test,y_train,y_test,pos,fr, fnum,filexs)
    period= time.time() - start 
    fr.write('number of particle {}:\n'.format(20))
    fr.write("best pos cost= "+str(cost)+"\nNumber of selected features:"+str(len([i for i in pos if i==1]))+"\nSelected features:"+str(pos))
    
    fr.write('\nFS, training, and testing time= '+str(period)+" seconds\n")
    fr.close()
filexs.to_excel("BPSO {}.xlsx".format(clas))

2021-12-08 19:35:06,706 - pyswarms.discrete.binary - INFO - Optimize for 100 iters with {'c1': 0.6, 'c2': 0.3, 'w': 0.4, 'k': 20, 'p': 2}
pyswarms.discrete.binary: 100%|███████████████████████████████████████████████████████████████|100/100, best_cost=0.237
2021-12-08 19:35:11,127 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 0.23728813559322035, best pos: [1 1 0 1 1 0 1 1 1 0 1 0 0 1 0 0 0 0 1 0]


cost: 0.23728813559322035
2
binary


IndexError: index 1 is out of bounds for axis 1 with size 1